In [1]:
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
import numpy as np
import cv2 as cv
import string
import os

In [2]:
BOARD_WIDTH  = 14 * 100 * 3
BOARD_HEIGHT = 14 * 100 * 3

MARGIN_CUTOFF_PERCENTAGE = 21 / 161

INITIAL_GRID = np.full((14, 14), -1)
INITIAL_GRID[6, 6:8] = [1, 2]
INITIAL_GRID[7, 6:8] = [3, 4]

OPERATIONS = [
    ["3", "#", "#", "#", "#", "#", "3", "3", "#", "#", "#", "#", "#", "3"],
    ["#", "2", "#", "#", "/", "#", "#", "#", "#", "/", "#", "#", "2", "#"],
    ["#", "#", "2", "#", "#", "-", "#", "#", "-", "#", "#", "2", "#", "#"],
    ["#", "#", "#", "2", "#", "#", "+", "*", "#", "#", "2", "#", "#", "#"],
    ["#", "/", "#", "#", "2", "#", "*", "+", "#", "2", "#", "#", "/", "#"],
    ["#", "#", "-", "#", "#", "#", "#", "#", "#", "#", "#", "-", "#", "#"],
    ["3", "#", "#", "*", "+", "#", "#", "#", "#", "*", "+", "#", "#", "3"],
    ["3", "#", "#", "+", "*", "#", "#", "#", "#", "+", "*", "#", "#", "3"],
    ["#", "#", "-", "#", "#", "#", "#", "#", "#", "#", "#", "-", "#", "#"],
    ["#", "/", "#", "#", "2", "#", "+", "*", "#", "2", "#", "#", "/", "#"],
    ["#", "#", "#", "2", "#", "#", "*", "+", "#", "#", "2", "#", "#", "#"],
    ["#", "#", "2", "#", "#", "-", "#", "#", "-", "#", "#", "2", "#", "#"],
    ["#", "2", "#", "#", "/", "#", "#", "#", "#", "/", "#", "#", "2", "#"],
    ["3", "#", "#", "#", "#", "#", "3", "3", "#", "#", "#", "#", "#", "3"],
]

DIGIT_MAP = {
    0: list(range(0, 10)),
    1: list(range(0, 10)),
    2: [0, 1, 4, 5, 7, 8],
    3: [0, 2, 5, 6],
    4: [0, 2, 5, 8, 9],
    5: [0, 4, 6],
    6: [0, 3, 4],
    7: [0, 2],
    8: [0, 1],
    9: [0]
}

OPERATIONS_MAP = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y if (y != 0 and x % y == 0) else None
}

COMPONENT_AREA_THRESHOLD = 2000
PADDING = 10

In [3]:
def show_image(title, image, save=False):
    resized_image = cv.resize(image, (0, 0), fx=0.2, fy=0.2)    
    cv.imshow(title, resized_image)
    
    if save:
        os.makedirs("debug", exist_ok=True)
        cv.imwrite(f"debug/{title}.jpg", resized_image)
    
    cv.waitKey(0)
    cv.destroyAllWindows()

In [4]:
def extract_board(image):
    hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    
    color_lb, color_ub = np.array([100, 50, 50]), np.array([140, 255, 255])
    mask = cv.inRange(hsv_image, color_lb, color_ub)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (7, 7))
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    mask = cv.equalizeHist(mask)

    # smooth edges
    blurred = cv.GaussianBlur(mask, (5, 5), 0)
    edges = cv.Canny(blurred, 50, 300)

    # show_image('mask', mask, True)
    # show_image('edges', edges, True)

    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_area = 0
    top_left = top_right = bottom_right = bottom_left = None

    for i in range(len(contours)):
        if len(contours[i]) > 3:
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis=1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
                
    width, height = BOARD_WIDTH, BOARD_HEIGHT

    """
    image_copy = image.copy()
    cv.circle(image_copy, tuple(top_left), 10, (0, 255, 0), -1)
    cv.circle(image_copy, tuple(top_right), 10, (0, 255, 0), -1)
    cv.circle(image_copy, tuple(bottom_left), 10, (0, 255, 0), -1)
    cv.circle(image_copy, tuple(bottom_right), 10, (0, 255, 0), -1)
    show_image("detected corners", image_copy)
    """

    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype="float32")
    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)

    return cv.warpPerspective(image, M, (width, height))

In [5]:
def extract_grid(image, width, height, margin_percentage):
    x_cutoff, y_cutoff = int(width * margin_percentage), int(height * margin_percentage)
    return image[y_cutoff:height - y_cutoff, x_cutoff:width - x_cutoff]

In [6]:
def grid_lines(shape):
    height, width = shape[0], shape[1]

    lines_horizontal=[]
    for i in range(0,height+1, int(height // 14)):
        l=[]
        l.append((0,i))
        l.append((height,i))
        lines_horizontal.append(l)
        
    lines_vertical=[]
    for i in range(0,width+1, int(width // 14)):
        l=[]
        l.append((i,0))
        l.append((i,width))
        lines_vertical.append(l)
        
    return lines_horizontal, lines_vertical

In [7]:
def add_grid_lines(grid):
    lines_horizontal, lines_vertical = grid_lines(grid.shape)
    
    for line in lines_vertical[1:-1]: 
        cv.line(grid, line[0], line[1], (0, 0, 0), PADDING)
    for line in  lines_horizontal[1:-1]: 
        cv.line(grid, line[0], line[1], (0, 0, 0), PADDING)
        
    return grid

In [8]:
def binary_grid(grid):
    grid = cv.cvtColor(grid, cv.COLOR_BGR2GRAY)
    grid_m_blur = cv.medianBlur(grid, 5)
    grid_g_blur = cv.GaussianBlur(grid_m_blur, (0, 0), 5)
    
    grid_sharpened = cv.addWeighted(grid_m_blur, 1.2, grid_g_blur, -0.8, 0)    
    _, binary_grid = cv.threshold(grid, 65, 255, cv.THRESH_BINARY_INV)
    
    # show_image('binary grid', binary_grid, True)
    return cv.dilate(binary_grid, np.ones((6, 6), np.uint8), iterations=1)

In [9]:
def clean_patch(patch):
    if np.mean(patch) < 10:
        return np.zeros_like(patch)
    
    num_labels, labels, stats, _ = cv.connectedComponentsWithStats(patch, connectivity=8)
    
    valid_components = []
    for i in range(1, num_labels):
        area = stats[i, cv.CC_STAT_AREA]
        height = stats[i, cv.CC_STAT_HEIGHT]
        width = stats[i, cv.CC_STAT_WIDTH]
        
        if area >= COMPONENT_AREA_THRESHOLD and height > width:
            valid_components.append(i)

    if not valid_components:
        return np.zeros_like(patch)

    largest_components = sorted(valid_components,
                                key=lambda i: stats[i, cv.CC_STAT_AREA], 
                                reverse=True)[:2]

    return np.where(np.isin(labels, largest_components), patch, 0)

In [10]:
def clean_grid(grid):
    lines_horizontal, lines_vertical = grid_lines(grid.shape)    

    new_grid = np.zeros_like(grid)
    for i in range(0, 14):
        for j in range(0, 14):
            y_min = lines_vertical[j][0][0] + PADDING
            y_max = lines_vertical[j + 1][1][0] - PADDING
            x_min = lines_horizontal[i][0][1] + PADDING
            x_max = lines_horizontal[i + 1][1][1] - PADDING
            
            patch = grid[x_min:x_max, y_min:y_max].copy()
            new_grid[x_min:x_max, y_min:y_max] = clean_patch(patch)

    return new_grid

In [11]:
def classify_number(patch):
   num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(patch, connectivity=8)
   valid_components = range(1, num_labels)
  
   if not valid_components:
       return -1
  
   largest_components = sorted(valid_components, key=lambda i: stats[i, cv.CC_STAT_AREA], reverse=True)[:2]
   largest_components.sort(key=lambda i: stats[i, cv.CC_STAT_LEFT])
   
   if not largest_components:
       return -1
  
   number = 0
   for i in largest_components:
       x, y, w, h = stats[i, cv.CC_STAT_LEFT], stats[i, cv.CC_STAT_TOP], stats[i, cv.CC_STAT_WIDTH], stats[i, cv.CC_STAT_HEIGHT]
      
       best_similarity, best_digit, best_template = -1, -1, None
       for digit in DIGIT_MAP[number]:
           template = cv.imread(f"digits/{digit}.jpg", cv.IMREAD_GRAYSCALE)
           template_resized = cv.resize(template, (w, h))
          
           component = patch[y:y+h, x:x+w]
           similarity = (1 + ssim(template_resized, component)) / 2
           if similarity > best_similarity:
               best_similarity = similarity
               best_template = component
               best_digit = digit
      
       number = number * 10 + best_digit
  
   #show_image(f'{number}', patch)
   return number

In [12]:
def extract_information(grid, grid_old):
    lines_horizontal, lines_vertical = grid_lines(grid.shape)

    best_white, best_patch = -1, None 
    selected_row, selected_col = -1, -1

    for i in range(0, 14):
        for j in range(0, 14):
            if grid_old[i, j] == -1:
                y_min = lines_vertical[j][0][0]
                y_max = lines_vertical[j + 1][1][0]
                x_min = lines_horizontal[i][0][1]
                x_max = lines_horizontal[i + 1][1][1]

                patch = grid[x_min:x_max, y_min:y_max]
                white_count = np.sum(patch == 255)

                if white_count > best_white:
                    best_white = white_count
                    best_patch = patch
                    selected_row = i
                    selected_col = j

    return selected_row, selected_col, classify_number(best_patch)

In [13]:
def predict_board_configuration(grid):
    lines_horizontal, lines_vertical = grid_lines(grid.shape)

    configuration = np.full((14, 14), -1)
    for i in range(0, 14):
        for j in range(0, 14):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]

            patch = grid[x_min:x_max, y_min:y_max]
            if np.mean(patch) > 10:
                    configuration[i][j] = classify_number(patch)

    return configuration

In [14]:
def process_image(image):
    transformations = [
        (extract_board, 'extracted-board'),
        (lambda board: extract_grid(board, BOARD_WIDTH, BOARD_HEIGHT, MARGIN_CUTOFF_PERCENTAGE), 'extracted-grid'),
        (add_grid_lines, 'grid-with-lines'),
        (binary_grid, 'binary-grid'),
        (clean_grid, 'cleaned-grid')
    ]
    
    for transform, title in transformations:
        image = transform(image)
        # show_image(title, image, True)
    
    return image

In [15]:
def valid_moves(grid, row, col):
    moves = [
        [(row - 1, col), (row - 2, col)],
        [(row + 1, col), (row + 2, col)],
        [(row, col - 1), (row, col - 2)],
        [(row, col + 1), (row, col + 2)],
    ]

    def is_valid_move(move):
        return all(0 <= r < 14 and 0 <= c < 14 and grid[r, c] != -1 for r, c in move)

    return [move for move in moves if is_valid_move(move)]

In [16]:
def calculate_score(grid, row, col, value, verbose=False):
    operation = OPERATIONS[row][col]
    count = 0
    
    for move in valid_moves(grid, row, col): 
        x = grid[move[0][0], move[0][1]]
        y = grid[move[1][0], move[1][1]]
        
        operations = [OPERATIONS_MAP[operation]] if operation in OPERATIONS_MAP.keys() else OPERATIONS_MAP.values()
        for function in operations:
            if function(x, y) == value or function(y, x) == value: 
                count += 1
                break
    
    return count * value * (int(operation) if operation in "23" else 1)

In [17]:
def predict(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for game in range(1, 5):
        print(f"Game {game}")
        turn_index, player1_score, player2_score = 1, 0, 0
        scores, turns = [], []
        
        turns_file = os.path.join(input_dir, f"{game}_turns.txt")
        with open(turns_file, 'r') as f:
            turns = [
                (int(line.split()[0].replace('Player', '')), int(line.split()[1]))
                for line in f.readlines()
            ]
            turns.append((turns[-1][0], 51))
        
        grid = INITIAL_GRID.copy()
        current_score = 0
        turn_results = []
        
        for turn in tqdm(range(1, 51)):
            turn_str = str(turn).zfill(2)
            image_path = os.path.join(input_dir, f"{game}_{turn_str}.jpg")
            
            try:
                processed_image = process_image(cv.imread(image_path))
                row, col, value = extract_information(processed_image, grid)
                grid[row][col] = value
                turn_results.append(f"{row+1}{string.ascii_uppercase[col]} {value}")
                current_score += calculate_score(grid, row, col, value)
            except Exception as e:
                print("Exception: ", str(e))
                turn_results.append(f"ERROR at turn {turn}")
            
            if turn_index < len(turns) and turns[turn_index][1] == turn + 1:
                scores.append(f"Player{turns[turn_index-1][0]} {turns[turn_index-1][1]} {current_score}")
                turn_index += 1
                current_score = 0
        
        for i, result in enumerate(turn_results, start=1):
            turn_str = str(i).zfill(2)
            turn_output_file = os.path.join(output_dir, f"{game}_{turn_str}.txt")
            try:
                with open(turn_output_file, 'w') as f:
                    f.write(result)
            except Exception as e:
                with open(turn_output_file, 'a') as f:
                    print(str(e))
                    f.write(f"ERROR writing file\n")
                    
        try:
            turns_output_file = os.path.join(output_dir, f"{game}_scores.txt")
            with open(turns_output_file, 'w') as f:
                f.write("\n".join(scores))
        except: 
            pass

In [18]:
predict(input_dir='evaluation/test', output_dir='evaluation/predictions')

Game 1


100%|██████████| 50/50 [00:15<00:00,  3.21it/s]


Game 2


100%|██████████| 50/50 [00:16<00:00,  3.00it/s]


Game 3


100%|██████████| 50/50 [00:16<00:00,  2.99it/s]


Game 4


100%|██████████| 50/50 [00:16<00:00,  3.10it/s]
